In [52]:
!pip3 install selenium

In [53]:
import pandas as pd
import numpy as np

## 1. Run Selenium and BeautifulSoup to scrape S&P Global ESG Scores page

In [54]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome("/chromedriver", options=chrome_options)

driver.get("https://www.spglobal.com/esg/scores/")
time.sleep(3)
click_accept = driver.find_elements(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')
click_accept[0].click()

historical_scores = {}
company_data = {}

# Key: Ticker, Value: Official Company Name (Based on S&P Global website)
med_companies = {
    "ABT" :"Abbott Laboratories",
    "BAX" : "Baxter International Inc.",
    "BDX" : "Becton, Dickinson and Company",
    "BSX" : "Boston Scientific Corporation",
    "XRAY" :"DENTSPLY SIRONA Inc",
    "DXCM" :"DexCom, Inc.",
    "EW" :"Edwards Lifesciences Corporation",
    "HOLX" :"Hologic, Inc.",
    "IDXX" :"IDEXX Laboratories",
    "PODD" :"Insulet Corporation",
    "ISRG" :"Intuitive Surgical",
    "MDT" :"Medtronic plc",
    "RMD" :"ResMed Inc.",
    "STE" :"STERIS plc",
    "SYK" :"Stryker Corporation",
    "TFX" :"Teleflex",
    "ZBH" :"Zimmer Biomet Holdings",
    "COO" :"The Cooper Companies, Inc.",
}


i = 1
for ticker, name in med_companies.items():
    if i == 1:
        search_box = driver.find_elements(By.XPATH, "/html/body/div[3]/div[11]/div/div[3]/div/div/div[2]/div[1]/input")
        search_box[0].send_keys(name)
    else:
        search_box = driver.find_elements(By.XPATH, "/html/body/div[3]/div[1]/div[3]/div/div/div[2]/div[1]/input")
        search_box[0].send_keys(name)
    time.sleep(1)
    search_box[0].send_keys(Keys.ENTER)
    time.sleep(5)

    try:
        ESG_score = driver.find_elements(By.XPATH, '//*[@id="esg-score"]')[0].text
        industry = driver.find_elements(By.XPATH, '//*[@id="company-industry"]')[0].text
        company_data[ticker] = industry
        time.sleep(3)

        historical_scores[ticker] = {}
        historical_scores[ticker]['ESG Scores By Year'] = []
        historical_scores[ticker]['ESG Scores By Category'] = {}
        historical_scores[ticker]['Company Performance'] = []
        historical_scores[ticker]['Industry Best Performance'] = []
        historical_scores[ticker]['Industry Mean Performance'] = []

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        historical_chart = soup.find('g', {'class': 'highcharts-markers highcharts-series-0 highcharts-line-series highcharts-tracker'})
        points = historical_chart.find_all('path', {'class':'highcharts-point'})
        for point in points:
            label = point.get('aria-label')
            historical_scores[ticker]['ESG Scores By Year'].append(label)

        company_perf = soup.find('g', {'class' : 'highcharts-markers highcharts-series-0 highcharts-area-series highcharts-tracker'})
        company_points = company_perf.find_all('path', {'class': 'highcharts-point'})
        for point in company_points:
            label = point.get('aria-label')
            historical_scores[ticker]['Company Performance'].append(label)

        industry_mean = soup.find('g', {'class' : 'highcharts-markers highcharts-series-2 highcharts-line-series highcharts-tracker'})
        industry_mean_points = industry_mean.find_all('path', {'class': 'highcharts-point'})
        for point in industry_mean_points:
            label = point.get('aria-label')
            historical_scores[ticker]['Industry Mean Performance'].append(label)

        industry_best = soup.find('g', {'class' : 'highcharts-markers highcharts-series-1 highcharts-line-series highcharts-tracker'})
        industry_best_points = industry_best.find_all('path', {'class': 'highcharts-point'})
        for point in industry_best_points:
            label = point.get('aria-label')
            historical_scores[ticker]['Industry Best Performance'].append(label)

        environmental_score_div = soup.find('div', {'class': 'dimention-chart1'})
        environmental_scores = environmental_score_div.find_all('li')
        historical_scores[ticker]['ESG Scores By Category']['Environmental'] = []
        for score in environmental_scores:
            historical_scores[ticker]['ESG Scores By Category']['Environmental'].append(score.text)

        social_score_div = soup.find('div', {'class': 'dimention-chart2'})
        social_scores = social_score_div.find_all('li')
        historical_scores[ticker]['ESG Scores By Category']['Social'] = []
        for score in social_scores:
            historical_scores[ticker]['ESG Scores By Category']['Social'].append(score.text)

        governance_score_div = soup.find('div', {'class': 'dimention-chart3'})
        governance_scores = governance_score_div.find_all('li')
        historical_scores[ticker]['ESG Scores By Category']['Governance'] = []
        for score in governance_scores:
            historical_scores[ticker]['ESG Scores By Category']['Governance'].append(score.text)

    except:
        if i == 1:
            search_box = driver.find_elements(By.XPATH, "/html/body/div[3]/div[11]/div/div[3]/div/div/div[2]/div[1]/input")
            search_box[0].clear()
        else:
            search_box = driver.find_elements(By.XPATH, "/html/body/div[3]/div[1]/div[3]/div/div/div[2]/div[1]/input")
            search_box[0].clear()
        print(f"!!!!!!!!!! unable to scrape {ticker} !!!!!!!!!!")
    i += 1

driver.close()
print(f"##### SCRAPING COMPLETED #####")

/var/folders/27/xh3tm0h964n5gjzv1brx5vrc0000gn/T/ipykernel_8940/1591206404.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/chromedriver", options=chrome_options)


##### SCRAPING COMPLETED #####


In [55]:
# Helper Function to Process Scrapped Data
def process_historical_scores(scores):
    """
    Takes in scrapped data and extract Historical ESG Scores, Company Performance in various categories,
    and Industry Performances (Mean and Best) in the same categories.

    Returns historical_ESG_scores -> dict, ESG_component_scores -> dict, company_performances -> dict
    """
    historical_ESG_scores = {}
    ESG_component_scores = {}
    company_performances = {}

    for ticker, scrapped_data in scores.items():
        historical_ESG_scores[ticker] = {}
        company_performances[ticker] = {}
        ESG_component_scores[ticker] = {}

        for data in scrapped_data['ESG Scores By Year']:
            split = data.split(", ")
            year = split[0].split(". ")[-1]
            esg_score = split[-1].split(".")[0]
            historical_ESG_scores[ticker][int(year)] = int(esg_score)
        
        for component, data in scrapped_data['ESG Scores By Category'].items():
            ESG_component_scores[ticker][component] = {}
            ESG_component_scores[ticker][component]['Company'] = int(data[0].split(" ")[-1])
            ESG_component_scores[ticker][component]['Industry Mean'] = int(data[1].split(" ")[-1])
            ESG_component_scores[ticker][component]['Industry Best'] = int(data[2].split(" ")[-1])

        for data in scrapped_data['Company Performance']:
            if "/ " in data: # to handle "Information Security/ Cybersecurity & System Availability"
                data = data.replace("/ ", " ")
            split = data.split(", ")
            category = split[0].split(". ")[-1]
            perf_score = split[-1].split(".")[0]
            if category not in company_performances[ticker].keys():
                company_performances[ticker][category] = {}    
            company_performances[ticker][category]['Company'] = int(perf_score)

        for data in scrapped_data['Industry Best Performance']:
            if "/ " in data: # to handle "Information Security/ Cybersecurity & System Availability"
                data = data.replace("/ ", " ")
            split = data.split(", ")
            category = split[0].split(". ")[-1]
            perf_score = split[-1].split(".")[0]
            if category not in company_performances[ticker].keys():
                company_performances[ticker][category] = {}  
            company_performances[ticker][category]['Industry Best'] = int(perf_score)

        for data in scrapped_data['Industry Mean Performance']:
            if "/ " in data: # to handle "Information Security/ Cybersecurity & System Availability"
                data = data.replace("/ ", " ")
            split = data.split(", ")
            category = split[0].split(". ")[-1]
            perf_score = split[-1].split(".")[0]
            if category not in company_performances[ticker].keys():
                company_performances[ticker][category] = {}  
            company_performances[ticker][category]['Industry Mean'] = int(perf_score)

    return historical_ESG_scores, ESG_component_scores, company_performances

In [56]:
historical_ESG_scores, ESG_component_scores, company_performances = process_historical_scores(historical_scores)

In [57]:
component_dict = {}
for ticker, data in ESG_component_scores.items():
    component_dict[ticker] = pd.DataFrame.from_dict(data, orient='index')

In [58]:
performance_dict = {}
for ticker, data in company_performances.items():
    performance_dict[ticker] = pd.DataFrame.from_dict(data, orient='index').sort_index()
performance_dict['ABT']

,Company,Industry Best,Industry Mean
Business Ethics,77,91,32
Health Outcome Contribution,100,100,7
Human Capital Development,100,100,22
Innovation Management,84,100,5
Operational Eco-Efficiency,96,97,14
Product Quality & Recall Management,63,100,45
Supply Chain Management,95,95,9
Talent Attraction & Retention,88,88,9


### Exploratory Data Analysis 

In [59]:
# Scrape Criteria Topics Mappings from S&P Global Website
chrome_options = Options()
chrome_options.add_argument('--headless')
driver = webdriver.Chrome("/chromedriver", options=chrome_options)

driver.get("https://www.spglobal.com/esg/scores/")
time.sleep(3)
criteria_list_container = driver.find_elements(By.XPATH, '/html/body/div[3]/div[17]/div')[0]
headers = criteria_list_container.find_elements(By.CLASS_NAME, 'item__header-title')
topics = criteria_list_container.find_elements(By.CLASS_NAME, 'item__content__list')

criteria_topic_mappings = {}
for social_topic in topics[0].text.split("\n"):
    criteria_topic_mappings[social_topic] = headers[0].text.split("\n")[0]

for environmental_topic in topics[1].text.split("\n"):
    criteria_topic_mappings[environmental_topic] = headers[1].text.split("\n")[0] + "s"
    
for governance_topic in topics[2].text.split("\n"):
    criteria_topic_mappings[governance_topic] = headers[2].text.split("\n")[0] + " Dimensions"

/var/folders/27/xh3tm0h964n5gjzv1brx5vrc0000gn/T/ipykernel_8940/1863327667.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/chromedriver", options=chrome_options)


In [60]:
criteria_topic_mappings

{'Addressing Cost Burden': 'Social Dimensions',
 'Asset Closure Management': 'Social Dimensions',
 'Corporate Citizenship & Philanthropy': 'Social Dimensions',
 'Financial Inclusion': 'Social Dimensions',
 'Health Outcome Contribution': 'Social Dimensions',
 'Human Capital Development': 'Social Dimensions',
 'Human Rights': 'Social Dimensions',
 'Labor Practice Indicators': 'Social Dimensions',
 'Local Impact of Business Operations': 'Social Dimensions',
 'Occupational Health & Safety': 'Social Dimensions',
 'Partnerships Towards Sustainable Healthcare': 'Social Dimensions',
 'Passenger Safety': 'Social Dimensions',
 'Responsibility of Content': 'Social Dimensions',
 'Social Impacts on Communities': 'Social Dimensions',
 'Social Integration & Regeneration': 'Social Dimensions',
 'Social Reporting': 'Social Dimensions',
 'Stakeholder Engagement': 'Social Dimensions',
 'Strategy to Improve Access to Drugs or Products': 'Social Dimensions',
 'Talent Attraction & Retention': 'Social Dimens

In [61]:

mapping_ESG = {}
for topic in list(performance_dict["MDT"].index):
    if topic in criteria_topic_mappings:
        mapping_ESG[topic] = criteria_topic_mappings[topic]
    else:
        if topic == "Business Ethics":
            mapping_ESG[topic] = "Social Dimension"
        elif topic == "Risk Management":
            mapping_ESG[topic] = "Governance & Economic Dimension"
mapping_ESG

{'Business Ethics': 'Social Dimension',
 'Health Outcome Contribution': 'Social Dimensions',
 'Human Capital Development': 'Social Dimensions',
 'Innovation Management': 'Governance & Economic Dimensions',
 'Operational Eco-Efficiency': 'Environmental Dimensions',
 'Product Quality & Recall Management': 'Governance & Economic Dimensions',
 'Supply Chain Management': 'Governance & Economic Dimensions',
 'Talent Attraction & Retention': 'Social Dimensions'}

In [62]:
historical_esg_scores_df = pd.DataFrame.from_dict(historical_ESG_scores)
historical_esg_scores_df

,ABT,BAX,BDX,BSX,XRAY,DXCM,EW,HOLX,IDXX,PODD,ISRG,MDT,RMD,STE,SYK,TFX,ZBH,COO
2018,82,71,52,34,22,10,69,22,17,12,12,61,19,13,14,14,28,11
2019,85,74,54,44,21,11,74,36,21,10,12,67,19,14,18,17,32,11
2020,86,76,48,51,22,11,73,47,18,15,14,70,25,19,24,18,33,10
2021,80,73,46,59,26,33,75,50,24,17,18,68,41,23,36,19,34,17
2022,81,70,45,62,27,34,75,52,29,20,20,65,42,25,42,29,29,27


In [63]:
ESG_component_scores

{'ABT': {'Environmental': {'Company': 73,
   'Industry Mean': 12,
   'Industry Best': 74},
  'Social': {'Company': 90, 'Industry Mean': 13, 'Industry Best': 90},
  'Governance': {'Company': 74, 'Industry Mean': 27, 'Industry Best': 78}},
 'BAX': {'Environmental': {'Company': 69,
   'Industry Mean': 12,
   'Industry Best': 74},
  'Social': {'Company': 66, 'Industry Mean': 13, 'Industry Best': 90},
  'Governance': {'Company': 75, 'Industry Mean': 27, 'Industry Best': 78}},
 'BDX': {'Environmental': {'Company': 51,
   'Industry Mean': 12,
   'Industry Best': 74},
  'Social': {'Company': 42, 'Industry Mean': 13, 'Industry Best': 90},
  'Governance': {'Company': 46, 'Industry Mean': 27, 'Industry Best': 78}},
 'BSX': {'Environmental': {'Company': 57,
   'Industry Mean': 12,
   'Industry Best': 74},
  'Social': {'Company': 69, 'Industry Mean': 13, 'Industry Best': 90},
  'Governance': {'Company': 57, 'Industry Mean': 27, 'Industry Best': 78}},
 'XRAY': {'Environmental': {'Company': 27,
   'I

In [64]:
combined_ESG_component = {}
for company, data in ESG_component_scores.items():
    if 'Environmental' not in combined_ESG_component:
        combined_ESG_component['Environmental'] = {}
        
    if 'Social' not in combined_ESG_component:
        combined_ESG_component['Social'] = {}
        
    if 'Governance' not in combined_ESG_component:
        combined_ESG_component['Governance'] = {}
        
    combined_ESG_component['Environmental'][company] = data['Environmental']['Company']
    combined_ESG_component['Social'][company] = data['Social']['Company']
    combined_ESG_component['Governance'][company] = data['Governance']['Company']

pd.DataFrame.from_dict(combined_ESG_component, orient='index')
    

,ABT,BAX,BDX,BSX,XRAY,DXCM,EW,HOLX,IDXX,PODD,ISRG,MDT,RMD,STE,SYK,TFX,ZBH,COO
Environmental,73,69,51,57,27,15,71,22,14,2,8,55,37,15,34,4,34,13
Social,90,66,42,69,18,22,74,56,28,9,9,67,34,18,33,32,23,23
Governance,74,75,46,57,37,51,77,56,34,36,35,66,52,36,53,31,35,35


In [65]:
combined_company_performance = {}
for company, data in company_performances.items():
    for topic, scores in data.items():
        if topic not in combined_company_performance:
            combined_company_performance[topic] = {}
        combined_company_performance[topic][company] = data[topic]['Company']
pd.DataFrame.from_dict(combined_company_performance, orient='index')

,ABT,BAX,BDX,BSX,XRAY,DXCM,EW,HOLX,IDXX,PODD,ISRG,MDT,RMD,STE,SYK,TFX,ZBH,COO
Business Ethics,77,78,54,67,60,64,78,60,34,35,33,78,72,62,41,46,43,49
Health Outcome Contribution,100,70,38,100,26,0,88,91,26,0,0,18,40,0,64,26,28,0
Human Capital Development,100,51,13,83,24,30,85,56,73,0,0,70,39,12,30,50,29,13
Innovation Management,84,61,18,37,0,62,80,65,0,0,0,84,8,0,8,0,0,0
Operational Eco-Efficiency,96,77,58,61,9,0,91,60,43,8,1,85,56,5,38,0,27,8
Product Quality & Recall Management,63,76,30,28,45,84,89,81,45,80,52,20,100,33,81,17,56,32
Supply Chain Management,95,87,58,88,9,21,64,41,17,13,10,85,34,38,72,18,18,15
Talent Attraction & Retention,88,58,35,61,7,9,74,50,14,8,5,80,26,10,19,6,7,7


### Exporting Historical ESG Scores

In [66]:
historical_ESG_scores_df = pd.DataFrame.from_dict(historical_ESG_scores)
historical_ESG_scores_df.to_csv("Historical_ESG_Scores/Healthcare_Companies_Historical_ESG_Scores.csv")

### Exporting ESG Component Score and Company Performance Scores

In [67]:
companies = list(ESG_component_scores.keys())
for company in companies:
    company_df = pd.DataFrame.from_dict(ESG_component_scores[company])
    company_perf_df = pd.DataFrame.from_dict(company_performances[company])
    company_df.to_csv(f"ESG_Component_Scores/{company}_ESG_Component.csv")
    company_perf_df.to_csv(f"Company_Performances/{company}_Performance.csv")